[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/anytko/ecospat/blob/main/docs/examples/range_edges.ipynb)

## Categorizing historical and modern range edges 

In [ ]:
import ecospat.ecospat as ecospat_full
from ecospat.stand_alone_functions import (
    get_species_code_if_exists,
    merge_touching_groups,
    assign_polygon_clusters,
    classify_range_edges,
    update_polygon_categories,
    get_start_year_from_species,
    fetch_gbif_data_with_historic,
    convert_to_gdf,
    process_gbif_data_pipeline,
    calculate_density,
    summarize_polygons_with_points,
    process_species_historical_range,
    analyze_species_distribution,
)

### Step-by-step historical

In [ ]:
# Range maps of over 600 North American tree species were created by Elbert L. Little, Jr. from 1971-1977
# First we need to load in the historical Little data for a tree species to an ecospat map

historic_map = ecospat_full.Map()
species_name = "Populus angustifolia"
code = get_species_code_if_exists(species_name)
historic_map.load_historic_data(species_name, add_to_map=True)
historic_map

In [ ]:
# Next we need to remove lakes and major bodies of water and merge touching polygons

range_no_lakes = historic_map.remove_lakes(historic_map.gdfs[code])

# We can update the buffer_distance parameter based what polygons we want to merge; 5000m is a good start

merged_polygons = merge_touching_groups(range_no_lakes, buffer_distance=5000)

merged_polygons.plot()

In [ ]:
# Finally, we can classify the range edges of the historical range

# Identifies large core polygons
clustered_polygons, largest_polygons = assign_polygon_clusters(merged_polygons)

# Classifies range edges based on latitudinal and longitudinal position to core polygons
classified_polygons = classify_range_edges(clustered_polygons, largest_polygons)

# Updates polygon categories for polygons on islands
updated_polygon = update_polygon_categories(largest_polygons, classified_polygons)

updated_polygon.plot(column="category", legend=True, figsize=(10, 12))

In [ ]:
# We can also plot these polygons on an ecospat map
historical_map_poly = ecospat_full.Map()
historical_map_poly.add_range_polygons(updated_polygon)
historical_map_poly

### Step-by-step modern

In [ ]:
# First we need to fetch modern and historic GBIF data. Historic GBIF data will be used to calculate population density change.

# Let's retrieve the year the associated little map was published for this species
start_year = get_start_year_from_species(species_name)
start_year = int(start_year)

# Now we will pull 1000 GBIF occurrences from 2025 backwards and from 1976 (start year) backwards
data = fetch_gbif_data_with_historic(
    species_name,
    limit=1000,
    start_year=start_year,
    end_year=2025,
    continent="north_america",
)
modern_data = data["modern"]
historic_data = data["historic"]

# Finally, we convert this raw GBIF data into a gdf
historic_gdf = convert_to_gdf(historic_data)
modern_gdf = convert_to_gdf(modern_data)

# As an example, we will view the first few rows of the modern GBIF gdf
modern_gdf.head()

In [ ]:
import matplotlib.pyplot as plt

# Now we will need to processes this raw GBIF data in order to classify range edges

classified_modern = process_gbif_data_pipeline(
    modern_gdf,
    species_name=species_name,
    is_modern=True,
    end_year=2025,
    continent="north_america",
)

ax = classified_modern.plot(column="category", legend=True, figsize=(10, 12))
ax.set_title("Modern GBIF Range Edges")

classified_historic = process_gbif_data_pipeline(
    historic_gdf, is_modern=False, end_year=2025, continent="north_america"
)

ax_historic = classified_historic.plot(column="category", legend=True, figsize=(10, 10))
ax_historic.set_title("Historic GBIF Range Edges")

In [ ]:
# We then need to calculate the density of points (or unique individuals per polygon)

classified_modern = calculate_density(classified_modern)
classified_historic = calculate_density(classified_historic)

summarized_modern = summarize_polygons_with_points(classified_modern)

summarized_modern.head()

In [ ]:
# Finally, lets add these modern polygons to an ecospat map

modern_map_poly = ecospat_full.Map()
modern_map_poly.add_range_polygons(summarized_modern)
modern_map_poly

## Pipeline functions 

### Historical pipeline

In [ ]:
# Here we are going to generate the historic range map data
hist_pipeline = ecospat_full.Map()
hist_range = process_species_historical_range(
    new_map=hist_pipeline, species_name="Populus angustifolia"
)
hist_pipeline.add_range_polygons(hist_range)
hist_pipeline

### Modern GBIF pipeline

In [ ]:
classified_modern, classified_historic = analyze_species_distribution(
    "Populus angustifolia", record_limit=1000
)
classified_modern

In [ ]:
modern_pipeline_summary = summarize_polygons_with_points(classified_modern)
modern_pipeline_map = ecospat_full.Map()
modern_pipeline_map.add_range_polygons(modern_pipeline_summary)
modern_pipeline_map